In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3,5,7],  
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [30, 40],
    'n_jobs': [None, -1],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p':[2,1]}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(X_train, y_train)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30, 40],
                         'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_jobs': [None, -1], 'n_neighbors': [3, 5, 7],
                         'p': [2, 1], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}: 0.9183922682195829


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 4 45]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9411764705882354

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_metric,param_n_jobs,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002893,0.000199,0.019868,0.003020,auto,30,euclidean,None,3,2,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.867478,0.906252,0.869709,0.962636,0.924528,0.906121,0.035642,49
1,0.003556,0.001215,0.009266,0.001842,auto,30,euclidean,None,3,2,distance,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.847141,0.906252,0.814409,0.906935,0.924528,0.879853,0.041903,481
2,0.002607,0.001043,0.017919,0.002033,auto,30,euclidean,None,3,1,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.867478,0.906252,0.869709,0.962636,0.924528,0.906121,0.035642,49
3,0.002483,0.000356,0.008590,0.000652,auto,30,euclidean,None,3,1,distance,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.847141,0.906252,0.814409,0.906935,0.924528,0.879853,0.041903,481
4,0.003200,0.000653,0.017452,0.000794,auto,30,euclidean,None,5,2,uniform,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0.002528,0.000507,0.119411,0.006783,brute,40,minkowski,-1,5,1,distance,"{'algorithm': 'brute', 'leaf_size': 40, 'metri...",0.867478,0.906252,0.833323,0.925272,0.962264,0.898918,0.044838,241
572,0.003884,0.001468,0.122848,0.007513,brute,40,minkowski,-1,7,2,uniform,"{'algorithm': 'brute', 'leaf_size': 40, 'metri...",0.867478,0.886792,0.851527,0.944161,0.962264,0.902445,0.043310,145
573,0.002627,0.000849,0.129943,0.012378,brute,40,minkowski,-1,7,2,distance,"{'algorithm': 'brute', 'leaf_size': 40, 'metri...",0.847141,0.906252,0.851527,0.944161,0.943041,0.898425,0.042363,289
574,0.002719,0.000603,0.120532,0.008876,brute,40,minkowski,-1,7,1,uniform,"{'algorithm': 'brute', 'leaf_size': 40, 'metri...",0.867478,0.886792,0.850543,0.944161,0.962264,0.902248,0.043543,193


In [21]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age:34
Salary:40000
Sex Male 0 or 1:0


In [22]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
